In [ ]:
# 使用新的配置模块进行优化实验
from research.optuna_config import create_robust_study, safe_optimize
from tuning_pipeline import optuna_log_manager, tune_pipeline

# 方法1: 创建新的研究
study_name = "backtest_tuning_find_deap_v1_bar"

with optuna_log_manager.verbose_optimization():
    study = create_robust_study(
        study_name=study_name,
        storage_dir="optuna_storage",
        direction="maximize",
        n_startup_trials=5000,
        percentile_for_pruning=20.0,
    )

    # 使用安全优化函数
    safe_optimize(
        study=study,
        objective=tune_pipeline,
        n_trials=6000,
        n_jobs=1,
        gc_after_trial=True,
        show_progress_bar=False,
    )


In [2]:
for i in sorted(
    study.trials, key=lambda t: t.value if t.value is not None else 0, reverse=True
)[:5]:
    print(f"{i.number}: {i.value} {i.params}")

1309: 1.988581582635361 {'vol_1': 164, 'r_1': 158, 'r_2': 168, 'threshold': 0.6384869661329587}
330: 0.7572246164264725 {'vol_1': 51, 'r_1': 88, 'r_2': 287, 'threshold': 0.3044867289147951}
1059: 0.7393380408089698 {'vol_1': 134, 'r_1': 71, 'r_2': 60, 'threshold': 0.30590187549758413}
1149: 0.7204371472067219 {'vol_1': 71, 'r_1': 280, 'r_2': 272, 'threshold': 0.6918144674840744}
1243: 0.7039038747414911 {'vol_1': 162, 'r_1': 32, 'r_2': 267, 'threshold': 0.26764460678281365}


In [ ]:
# 方法2: 继续之前中断的实验
from research.optuna_config import OptunaStorageManager, safe_optimize
from tuning_pipeline import optuna_log_manager, tune_pipeline

# 使用存储管理器
manager = OptunaStorageManager("optuna_storage")

# 列出所有可用的研究
studies = manager.list_studies()
if studies:
    print("可用的研究:")
    for i, summary in enumerate(studies):
        print(f"  {i + 1}. {summary.study_name}: {summary.n_trials} 个试验")

        # 方法2a: 继续特定的研究
        study_name_to_continue = "backtest_tuning_main"  # 指定要继续的研究名称

        if manager.study_exists(study_name_to_continue):
            print(f"\n继续研究: {study_name_to_continue}")

            with optuna_log_manager.verbose_optimization():
                # 直接加载指定的研究
                study = manager.create_or_load_study(
                    study_name=study_name_to_continue, load_if_exists=True
                )

                # 显示当前状态
                info = manager.get_study_info(study_name_to_continue)
                print(f"研究信息: {info}")

                # 继续优化
                safe_optimize(
                    study=study,
                    objective=tune_pipeline,
                    n_trials=50000,
                    n_jobs=1,
                    gc_after_trial=True,
                )
        else:
            print(f"研究 '{study_name_to_continue}' 不存在")

        # 方法2b: 继续最新的研究
        # latest_study = manager.get_latest_study()
        # if latest_study:
        #     print(f"继续最新的研究: {latest_study.study_name}")

        #     with optuna_log_manager.verbose_optimization():
        #         safe_optimize(
        #             study=latest_study,
        #             objective=tune_pipeline,
        #             n_trials=50000,
        #             n_jobs=1,
        #             gc_after_trial=True,
        #         )
        # else:
        #     print("没有找到任何研究，请先运行第一个cell创建新研究")
else:
    print("没有找到任何研究，请先运行第一个cell创建新研究")

In [ ]:
import numpy as np

from tuning_pipeline import BacktestPipeline


def tune_pipeline():
    pipeline = BacktestPipeline("data/btc_1m.npy")
    n1 = 10
    n2 = 168
    n_entropy = 54
    pipeline.init_bar_container(n1, n2, n_entropy)
    pipeline.set_threshold(0.3166890909253018)

    pipeline.generate_merged_bar()
    pipeline.side_labeling()
    pipeline.get_df_feature()
    pipeline.side_feature_selection()
    side_auc = pipeline.train_side_model()

    meta_label = pipeline.meta_labeling()
    pipeline.meta_feature_selection()
    meta_f1, meta_precision, meta_recall = pipeline.train_meta_model()

    calmar_ratio = pipeline.backtest()

    mask_train = pipeline.df_feature.index < pipeline.meta_model_split_timestamp
    mask_test = pipeline.df_feature.index >= pipeline.meta_model_split_timestamp

    meta_label_train_count = np.unique(meta_label[mask_train], return_counts=True)
    meta_label_test_count = np.unique(meta_label[mask_test], return_counts=True)

    print(
        f"{side_auc = :.6f} {meta_f1 = :.6f} {meta_precision = :.6f} {meta_recall = :.6f} {calmar_ratio = :.6f} {meta_label_train_count = } {meta_label_test_count = }"
    )
    return calmar_ratio * side_auc**3 * meta_f1**2


tune_pipeline()